## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-03-51-00 +0000,wsj,BOJ Policy Should Align With Government Policy...,https://www.wsj.com/economy/boj-policy-should-...
1,2025-10-24-03-50-28 +0000,nyt,U.S. May Seek United Nations Mandate for Gaza ...,https://www.nytimes.com/2025/10/23/world/middl...
2,2025-10-24-03-47-00 +0000,wsj,Alaska Airlines Works to Restore Operations Af...,https://www.wsj.com/business/airlines/alaska-a...
3,2025-10-24-03-44-39 +0000,nypost,Coca-Cola recalls 3 popular sodas over concern...,https://nypost.com/2025/10/23/business/coca-co...
4,2025-10-24-03-41-08 +0000,nypost,"JB Pritzker accuses ICE of ‘racial profiling,’...",https://nypost.com/2025/10/23/us-news/jb-pritz...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2584/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,64
222,new,19
48,off,14
87,sanctions,14
88,russian,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
288,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,128
138,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...,127
265,2025-10-23-13-32-00 +0000,nyt,Trump Imposes Sanctions on Russian Oil Compani...,https://www.nytimes.com/2025/10/22/us/politics...,122
168,2025-10-23-18-09-53 +0000,nypost,"China, India curb Russian oil purchases after ...",https://nypost.com/2025/10/23/world-news/china...,121
147,2025-10-23-20-01-18 +0000,nypost,Here are the corporations and donors footing t...,https://nypost.com/2025/10/23/us-news/here-are...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
288,128,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...
114,54,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
12,52,2025-10-24-03-00-00 +0000,wsj,Will India and China Be Able to Resist U.S. Sa...,https://www.wsj.com/world/asia/india-china-tru...
88,48,2025-10-23-22-15-21 +0000,nypost,‘Mobbed-up’ NBA gambling suspect Nicholas ‘Fat...,https://nypost.com/2025/10/23/us-news/mobbed-u...
24,44,2025-10-24-01-40-54 +0000,nypost,White House official suggests married ex-GOP R...,https://nypost.com/2025/10/23/us-news/white-ho...
112,44,2025-10-23-21-29-34 +0000,latimes,"Trump calls off San Francisco 'surge,' but Eas...",https://www.latimes.com/california/story/2025-...
263,42,2025-10-23-13-34-07 +0000,nypost,"King Charles, Pope Leo pray together in rare m...",https://nypost.com/2025/10/23/world-news/king-...
84,40,2025-10-23-22-21-54 +0000,nypost,How mafiosos and NBA stars used rigged card sh...,https://nypost.com/2025/10/23/us-news/how-mafi...
221,39,2025-10-23-15-38-53 +0000,nypost,Trump pardons convicted Binance founder Changp...,https://nypost.com/2025/10/23/business/trump-p...
175,37,2025-10-23-18-06-16 +0000,nypost,French museum was robbed of 2K gold and silver...,https://nypost.com/2025/10/23/world-news/a-sec...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
